In [1]:
import string
from IPython.core.display import display, HTML
import itertools
import detectEnglish

In [2]:
alphabeth = list(string.ascii_lowercase + ' ' + string.digits)

In [3]:
def encrypt(text, gamma):
    text = text.lower()
    textLen = len(text)
    gammaLen = len(gamma)

    #Формируем ключевое слово(растягиваем гамму на длину текста)
    keyText = []
    for i in range(textLen // gammaLen):
        for symb in gamma:
            keyText.append(symb)
    for i in range(textLen % gammaLen):
        keyText.append(gamma[i])

    #Шифрование
    code = []
    for i in range(textLen):
        code.append(alphabeth[(alphabeth.index(text[i]) + alphabeth.index(keyText[i])) % len(alphabeth)])

    return code

In [4]:
def decrypt(code, gamma):
    codeLen = len(code)
    gammaLen = len(gamma)

    #Формируем ключевое слово(растягиваем гамму на длину текста)
    keyText = []
    for i in range(codeLen // gammaLen):
        for symb in gamma:
            keyText.append(symb)
    for i in range(codeLen % gammaLen):
        keyText.append(gamma[i])

    #Расшифровка
    text = []
    for i in range(codeLen):
        text.append(alphabeth[(alphabeth.index(code[i]) - alphabeth.index(keyText[i]) + len(alphabeth)) % len(alphabeth)]) 

    return text

In [5]:
text = 'hello my name is vlad and the weather is fine'
gamma = 'hello'

In [6]:
encrypted_text = ''.join(encrypt(text,gamma))
display(HTML('<h3>зашифрованный текст:</h3> '+ encrypted_text))

In [7]:
decrypted_text = ''.join(decrypt(encrypted_text, gamma))
display(HTML('<h3>дешифрованный текст:</h3> '+ decrypted_text))

In [8]:
def get_ic(s): #получение суммы для теста Фридмана (для одного элемента)
    return sum(map(lambda x: x*(x-1), [len(list(y)) for x,y in itertools.groupby(sorted(s))])) / len(s) / (len(s)-1)
 
def friedman(s, key_len): #получение индекса для зашифрованной строки и длинны гаммы
    return sum([get_ic(s[start::key_len]) for start in range(key_len)]) / key_len 

def get_gamma_length(): #получение первой подходящей длинны гаммы
    for key_len in range(1,len(alphabeth) + 1):
        if friedman(encrypted_text, key_len) > 0.053:
            return key_len
        
def get_gamma_arr(): #получение всего массива индексов для возможных длинн гаммы
    arr = []
    for key_len in range(1,len(alphabeth) + 1):
        try:
            arr.append({key_len : friedman(encrypted_text, key_len)})
        except:
            return arr
    return arr

In [9]:
gammaLen  = get_gamma_length()
for ind in get_gamma_arr():
    print(ind)
display(HTML('<h3>найденная длина гаммы: </h3>' + str(gammaLen)))

{1: 0.02727272727272727}
{2: 0.026821005081874648}
{3: 0.022222222222222223}
{4: 0.040151515151515146}
{5: 0.055555555555555546}
{6: 0.025793650793650796}
{7: 0.009523809523809523}
{8: 0.029166666666666667}
{9: 0.0}
{10: 0.04666666666666667}
{11: 0.0303030303030303}
{12: 0.05555555555555555}
{13: 0.02564102564102564}
{14: 0.0}
{15: 0.06666666666666667}
{16: 0.041666666666666664}
{17: 0.0784313725490196}
{18: 0.0}
{19: 0.017543859649122806}
{20: 0.06666666666666667}
{21: 0.0}
{22: 0.045454545454545456}


In [10]:
answer = str(input('поменять длину гаммы? (y/n)')).lower()
if answer == 'y':
    gammaLen = int(input('введите новую длину: '))
display(HTML('<h3>финальная длина гаммы: </h3>' + str(gammaLen)))

поменять длину гаммы? (y/n)


In [11]:
def hack(ciphertext): #взлом гамма-шифра
    fo = open('dictionary.txt')
    words = fo.readlines()
    fo.close()
    words_arr =[]

    for word in words:
        word = word.lower().strip()
        if len(word) != gammaLen : continue

        decryptedText = ''.join(decrypt(ciphertext, word))
        if detectEnglish.isEnglish(decryptedText, wordPercentage=40):
            return decryptedText


In [12]:
display(HTML('<h3>текст для взлома:</h3>' + encrypted_text))
display(HTML('<h3>предполагаемая длинна гаммы:</h3>' + str(gammaLen)))
display(HTML('<h3>результат:</h3>' + str(hack(encrypted_text))))